## Token filtering
the training is taking too long. we will filter for length

In [2]:
from datasets import load_dataset

code = load_dataset("open-r1/codeforces-cots", "solutions")["train"]

In [3]:
print(code)
print(code[0])

Dataset({
    features: ['id', 'aliases', 'contest_id', 'contest_name', 'contest_type', 'contest_start', 'contest_start_year', 'index', 'time_limit', 'memory_limit', 'title', 'description', 'input_format', 'output_format', 'examples', 'note', 'editorial', 'prompt', 'generation', 'finish_reason', 'api_metadata', 'interaction_format', 'messages'],
    num_rows: 47780
})
{'id': '1991/A', 'aliases': None, 'contest_id': '1991', 'contest_name': 'Pinely Round 4 (Div. 1 + Div. 2)', 'contest_type': 'CF', 'contest_start': 1722177300, 'contest_start_year': 2024, 'index': 'A', 'time_limit': 1.0, 'memory_limit': 256, 'title': 'Maximize the Last Element', 'description': 'You are given an array $$$a$$$ of $$$n$$$ integers, where $$$n$$$ is odd.\n\nIn one operation, you will remove two adjacent elements from the array $$$a$$$, and then concatenate the remaining parts of the array. For example, given the array $$$[4,7,4,2,9]$$$, we can obtain the arrays $$$[4,2,9]$$$ and $$$[4,7,9]$$$ by the operations

In [19]:
from transformers import AutoTokenizer

model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.chat_template = "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n    {%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {%- if message['role'] == 'assistant' %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n            {% generation %}\n            {{- message['content'] | trim + '<|eot_id|>' }}\n            {% endgeneration %}\n        {%- else %}\n            {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n        {%- endif %}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n        {% generation %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n            {%- endfor %}\n            {{- \")\" }}\n        {%- else %}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n        {% endgeneration %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}" #adapted for Llama with reasoning chat template, and generation tags
  

messages = [
    {"role": "user", "content": "hello"},
    {"role": "assistant", "content": "yo"}
]

print(
    tokenizer.apply_chat_template(
        messages, 
        
        # tokenize=False,
        
        return_assistant_tokens_mask=True,
        return_dict=True,
        )
    
    )

{'input_ids': [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 2675, 527, 264, 11190, 15592, 22103, 430, 5825, 1664, 5621, 1525, 291, 323, 11944, 14847, 13, 1472, 1176, 1781, 922, 279, 33811, 1920, 439, 459, 5419, 1647, 77828, 323, 1243, 3493, 279, 1217, 449, 279, 4320, 13, 40633, 304, 279, 2768, 3645, 25, 366, 27963, 397, 9522, 524, 27963, 397, 27, 9399, 397, 9522, 524, 9399, 29, 128009, 128006, 882, 128007, 271, 15339, 128009, 128006, 78191, 128007, 1432, 16417, 128009, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'assistant_masks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [26]:
conversation = [
    {"role": "user", "content": "Hello assistant"},
    {"role": "assistant", "content": "Hello user"},
    # {"role": "user", "content": "How are you?"},
    # {"role": "assistant", "content": "I'm good"},
    ]

tokenized_output = tokenizer.apply_chat_template(
    conversation,
    return_assistant_tokens_mask=True,
    return_dict=True,
    )
tokenized_output

{'input_ids': [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 2675, 527, 264, 11190, 15592, 22103, 430, 5825, 1664, 5621, 1525, 291, 323, 11944, 14847, 13, 1472, 1176, 1781, 922, 279, 33811, 1920, 439, 459, 5419, 1647, 77828, 323, 1243, 3493, 279, 1217, 449, 279, 4320, 13, 40633, 304, 279, 2768, 3645, 25, 366, 27963, 397, 9522, 524, 27963, 397, 27, 9399, 397, 9522, 524, 9399, 29, 128009, 128006, 882, 128007, 271, 9906, 18328, 128009, 128006, 78191, 128007, 1432, 9906, 1217, 128009, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'assistant_masks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
input_ids = tokenized_output['input_ids']
attention_mask = tokenized_output['attention_mask']
assistant_masks = tokenized_output['assistant_masks']
for elt in zip(input_ids, attention_mask, assistant_masks):
    input_id, attention_mas, assistant_mask = elt
    decoded = tokenizer.decode([input_id])
    print(attention_mas, assistant_mask, input_id, decoded)

1 0 128000 <|begin_of_text|>
1 0 128006 <|start_header_id|>
1 0 9125 system
1 0 128007 <|end_header_id|>
1 0 271 


1 0 38766 Cut
1 0 1303 ting
1 0 33025  Knowledge
1 0 2696  Date
1 0 25 :
1 0 6790  December
1 0 220  
1 0 2366 202
1 0 18 3
1 0 198 

1 0 15724 Today
1 0 2696  Date
1 0 25 :
1 0 220  
1 0 1627 26
1 0 10263  Jul
1 0 220  
1 0 2366 202
1 0 19 4
1 0 271 


1 0 2675 You
1 0 527  are
1 0 264  a
1 0 11190  helpful
1 0 15592  AI
1 0 22103  Assistant
1 0 430  that
1 0 5825  provides
1 0 1664  well
1 0 5621 -re
1 0 1525 ason
1 0 291 ed
1 0 323  and
1 0 11944  detailed
1 0 14847  responses
1 0 13 .
1 0 1472  You
1 0 1176  first
1 0 1781  think
1 0 922  about
1 0 279  the
1 0 33811  reasoning
1 0 1920  process
1 0 439  as
1 0 459  an
1 0 5419  internal
1 0 1647  mon
1 0 77828 ologue
1 0 323  and
1 0 1243  then
1 0 3493  provide
1 0 279  the
1 0 1217  user
1 0 449  with
1 0 279  the
1 0 4320  answer
1 0 13 .
1 0 40633  Respond
1 0 304  in
1 0 279  the
1 0 2768  following
1 0 3645  fo

In [28]:
len(tokenizer.apply_chat_template(code[0]['messages'], tokenize=True))

3449

In [29]:
print(len(code))

47780


In [30]:


lengths = []
from tqdm import tqdm
for i in tqdm(range(100)):
    messages = code[i]['messages']
    tokenized = tokenizer.apply_chat_template(messages, tokenize=True)
    length = len(tokenized)
    lengths.append(length)
print(lengths)
print(sum(lengths) / len(lengths))

100%|██████████| 100/100 [00:01<00:00, 54.14it/s]

[3449, 4421, 3204, 2925, 4805, 6173, 6880, 7342, 6753, 9097, 6535, 7455, 8918, 6359, 8204, 11165, 10921, 13042, 11626, 10662, 5524, 5901, 8057, 7062, 6010, 6231, 5679, 9652, 5177, 6224, 20260, 18700, 16100, 14991, 15097, 9686, 12207, 10077, 20550, 8790, 24662, 23515, 23742, 25072, 24533, 4925, 5070, 4971, 4663, 5468, 15375, 21716, 19502, 19049, 20486, 8022, 6973, 9543, 7080, 9658, 3720, 2204, 2416, 3222, 3975, 2967, 3444, 3020, 2995, 3979, 2875, 4137, 3287, 2354, 5901, 5955, 10217, 7458, 6528, 16341, 10835, 12946, 11909, 13315, 14462, 10970, 9477, 9970, 11108, 11625, 8451, 7901, 5886, 7265, 6140, 19052, 19634, 23283, 21480, 18465]
9971.05


In [ ]:
# check tokenized length
n = 4096
def filter_long_rows(example):
    messages = example['messages']
    tokenized = tokenizer.apply_chat_template(messages, tokenize=True)
    length = len(tokenized)
    return length <= n

In [9]:
first = code[8]
# print(first['messages'])
print(tokenizer.apply_chat_template(first['messages'], tokenize=False))
print(len(tokenizer.apply_chat_template(first['messages'], tokenize=True)))
filter_long_rows(first)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>
...
</think>
<answer>
...
</answer><|eot_id|><|start_header_id|>user<|end_header_id|>

You will be given a competitive programming problem. Please reason step by step about the solution, then provide a complete implementation in C++17.

Your solution must read input from standard input (cin), write output to standard output (cout).
Do not include any debug prints or additional output.

Put your final solution within a single code block:
```cpp
<your code here>
```

# Problem

You are given two set of points. The first set is determined by the equation A1x + B1y + C1 = 0, and the second one is determined by the equation A2x + B2y + C

False

In [10]:
code_filtered = code.filter(filter_long_rows, num_proc=4)

In [11]:
print(code_filtered)

Dataset({
    features: ['id', 'aliases', 'contest_id', 'contest_name', 'contest_type', 'contest_start', 'contest_start_year', 'index', 'time_limit', 'memory_limit', 'title', 'description', 'input_format', 'output_format', 'examples', 'note', 'editorial', 'prompt', 'generation', 'finish_reason', 'api_metadata', 'interaction_format', 'messages'],
    num_rows: 4148
})


In [12]:
86158/131396

0.6557125026637036

In [13]:
# code_filtered.push_to_hub("Neelectric/codeforces-cots_solutions_Llama3_4096toks")

In [14]:
# Neelectric/codeforces-cots_solutions_Llama3_16384toks